<a href="https://colab.research.google.com/github/sinchir0/lmsys/blob/main/lmsys/exp/exp007_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 目的
Concatをlargeで行う

In [ ]:
# path setting
EXP_NAME = "e007-use-concat-large-colab"
MODEL_NAME = "microsoft/deberta-v3-large"
COMPETITION_NAME = "lmsys"

DATA_PATH = "data"
DATASET_NAME = f"{EXP_NAME}-{MODEL_NAME.split('/')[-1]}"
MODEL_OUTPUT_PATH = f"trained_models/{EXP_NAME}"

# experiment parameter
DEBUG = False
TRAINING = True
UPLOAD_DATA_TO_S3 = True
UPLOAD_DATA_TO_KAGGLE = True
WANDB = True

# model parameter
TRAINING_MAX_LENGTH = 512
INFERENCE_MAX_LENGTH = 512
SEED = 42
VALID_DATA_SIZE = 0.3
EPOCH = 3
LR = 2e-05
TRAIN_BS = 128 # 32 # 16 # 8
GRAD_ACC_NUM = 256 # 128 # 256 # 32 # 16
EVAL_BS = 128 # 32 # 16 # 64 # 8
NUM_LABELS = 3

In [ ]:
!nvidia-smi

Thu Jul 11 21:30:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0              44W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
!python --version

Python 3.10.12


In [ ]:
def resolve_path(base_path: str) -> str:
    import os

    cwd = os.getcwd()
    print(cwd)
    if cwd == f"/notebooks":
        print("Jupyter Kernel By VSCode!")
        return f"/notebooks/{COMPETITION_NAME}/{base_path}"
    elif cwd == f"/notebooks/{COMPETITION_NAME}":
        print("nohup!")
        return base_path
    elif cwd == f"/notebooks/{COMPETITION_NAME}/{COMPETITION_NAME}/exp":
        print("Jupyter Lab!")
        return f"../../{base_path}"
    elif cwd == f"/content":
        print("Google Colab!")
        return f"/content/drive/MyDrive/Kaggle/{COMPETITION_NAME}/{base_path}"
    else:
        raise Exception("Unknown environment")


DATA_PATH = resolve_path(DATA_PATH)
print(DATA_PATH)
MODEL_OUTPUT_PATH = resolve_path(MODEL_OUTPUT_PATH)
print(MODEL_OUTPUT_PATH)

/content
Google Colab!
/content/drive/MyDrive/Kaggle/lmsys/data
/content
Google Colab!
/content/drive/MyDrive/Kaggle/lmsys/trained_models/e007-use-concat-large-colab


In [ ]:
def validate_dataset_name(dataset_name: str) -> None:
    if len(dataset_name) < 6 or len(dataset_name) > 50:
        raise Exception(
            f"データセットの文字列は6~50文字にしてください。現在{len(DATASET_NAME)}文字"
        )
    if "_" in dataset_name:
        raise Exception("datasetの名称に_の使用は禁止です")


validate_dataset_name(DATASET_NAME)

# install

In [ ]:
%pip install -qq polars==1.0.0
%pip install -qq transformers==4.42.3
%pip install -qq datasets==2.20.0
%pip install -qq evaluate==0.4.2
%pip install -qq seqeval==1.2.2
%pip install -qq accelerate==0.32.0
%pip install -qq python-dotenv==1.0.1
%pip install -qq wandb==0.17.4

# formatter
%pip install -qq black isort

%pip install -qq kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2

# import

In [ ]:
import os
import random
import ast
import json

import polars as pl
import numpy as np
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, LayerNorm, MSELoss
import wandb
from datasets import Dataset, DatasetDict, Value, concatenate_datasets, load_dataset, ClassLabel
from tokenizers import AddedToken
from tqdm.auto import tqdm
from scipy.special import softmax
from sklearn.metrics import log_loss
from transformers import (
    AutoConfig,
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    DebertaV2PreTrainedModel,
    Trainer,
    TrainingArguments,
)
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.models.deberta_v2.modeling_deberta_v2 import (
    ContextPooler,
    StableDropout,
    DebertaV2Model,
)

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"
NUM_PROC = os.cpu_count()

In [ ]:
import transformers
import datasets
import evaluate

assert transformers.__version__ == "4.42.3"
assert datasets.__version__ == "2.20.0"
assert evaluate.__version__ == "0.4.2"

In [ ]:
# Seed the same seed to all
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


seed_everything(SEED)

In [ ]:
from dotenv import load_dotenv

load_dotenv(f"{DATA_PATH}/.env")

True

# Wandb

In [ ]:
if WANDB:
    wandb.login(key=os.environ["WANDB_API_KEY"])
    wandb.init(project=COMPETITION_NAME, name=EXP_NAME)
    REPORT_TO = "wandb"
else:
    REPORT_TO = "none"

REPORT_TO

wandb: Currently logged in as: sinchir0. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


'wandb'

# Data Import & Preprocess

In [ ]:
train = (
    pl.read_csv(f"{DATA_PATH}/train.csv")
    .with_columns(
        pl.col("prompt").str.json_decode(),
        pl.col("response_a").str.json_decode(),
        pl.col("response_b").str.json_decode(),
    )
    .with_columns( # 長さの情報を追加する
        pl.col("prompt").map_elements(lambda x: len(x), return_dtype=pl.Int64).alias("len_prompt"),
        pl.col("response_a").map_elements(lambda x: len(x), return_dtype=pl.Int64).alias("len_response_a"),
        pl.col("response_b").map_elements(lambda x: len(x), return_dtype=pl.Int64).alias("len_response_b"),
    )
    .with_columns( # 最後のレスポンスのみを取得する
        pl.col("prompt").map_elements(lambda x: x[-1], return_dtype=pl.String).alias("last_prompt"),
        pl.col("response_a").map_elements(lambda x: x[-1], return_dtype=pl.String).alias("last_response_a"),
        pl.col("response_b").map_elements(lambda x: x[-1], return_dtype=pl.String).alias("last_response_b"),
    )
    .with_columns( # 最後のレスポンスがNoneの場合を空文字にする、約60件程度
        pl.col("last_response_a").fill_null(""),
        pl.col("last_response_b").fill_null(""),
    )
    .with_columns( # labelを付与する
        pl.when(pl.col("winner_model_a") == 1)
        .then(0)
        .when(pl.col("winner_model_b") == 1)
        .then(1)
        .when(pl.col("winner_tie") == 1)
        .then(2)
        .alias("label"),
    )
    .select( # 元のprompt, responseを削除する
        pl.exclude(["prompt", "response_a", "response_b"])
    )
)

In [ ]:
if DEBUG:
    train = train.head(100)

In [ ]:
train_dataset = Dataset.from_polars(train)

In [ ]:
train_dataset

Dataset({
    features: ['id', 'model_a', 'model_b', 'winner_model_a', 'winner_model_b', 'winner_tie', 'len_prompt', 'len_response_a', 'len_response_b', 'last_prompt', 'last_response_a', 'last_response_b', 'label'],
    num_rows: 57477
})

# Model

In [ ]:
# https://chatgpt.com/share/f7a30189-2ca6-4870-96e6-c06120bf4ca7

# class CustomDebertaModel(nn.Module):
#     def __init__(self, model_name):
#         super(CustomDebertaModel, self).__init__()
#         self.deberta = DebertaModel.from_pretrained(model_name)
#         self.classification_head = nn.Sequential(
#             nn.Linear(self.deberta.config.hidden_size * 2, 256),
#             nn.ReLU(),
#             nn.Linear(256, 1)  # Assuming binary classification
#         )

#     def forward(self, input_ids_a, attention_mask_a, input_ids_b, attention_mask_b):
#         outputs_a = self.deberta(input_ids=input_ids_a, attention_mask=attention_mask_a)
#         outputs_b = self.deberta(input_ids=input_ids_b, attention_mask=attention_mask_b)

#         cls_a = outputs_a.last_hidden_state[:, 0, :]  # CLS token
#         cls_b = outputs_b.last_hidden_state[:, 0, :]  # CLS token

#         combined = torch.cat((cls_a, cls_b), dim=1)
#         logits = self.classification_head(combined)
#         return logits

In [ ]:
# # https://github.com/sinchir0/automated_essay_scoring/blob/main/automated_essay_scoring/exp/exp030.ipynb

# # https://dev.classmethod.jp/articles/huggingface-usage-custom-model/
# # https://github.com/huggingface/transformers/blob/94b3f544a1f5e04b78d87a2ae32a7ac252e22e31/src/transformers/models/deberta_v2/modeling_deberta_v2.py#L1313
# class CustomDebertaSequenceClassification(DebertaV2PreTrainedModel):
#     def __init__(self, config):
#         super().__init__(config)

#         num_labels = getattr(config, "num_labels", 2)
#         self.num_labels = num_labels

#         self.deberta = DebertaV2Model(config)
#         self.pooler = ContextPooler(config)
#         output_dim = self.pooler.output_dim

#         self.classifier = nn.Linear(output_dim, num_labels)
#         drop_out = getattr(config, "cls_dropout", None)
#         drop_out = self.config.hidden_dropout_prob if drop_out is None else drop_out
#         self.dropout = StableDropout(drop_out)

#         # Initialize weights and apply final processing
#         self.post_init()

#     def get_input_embeddings(self):
#         return self.deberta.get_input_embeddings()

#     def set_input_embeddings(self, new_embeddings):
#         self.deberta.set_input_embeddings(new_embeddings)

#     def forward(
#         self,
#         input_ids: Optional[torch.Tensor] = None,
#         attention_mask: Optional[torch.Tensor] = None,
#         token_type_ids: Optional[torch.Tensor] = None,
#         # position_ids: Optional[torch.Tensor] = None,
#         inputs_embeds: Optional[torch.Tensor] = None,
#         labels: Optional[torch.Tensor] = None,
#         # output_attentions: Optional[bool] = None,
#         # output_hidden_states: Optional[bool] = None,
#         return_dict: Optional[bool] = None,
#     ) -> Union[Tuple, SequenceClassifierOutput]:
#         return_dict = (
#             return_dict if return_dict is not None else self.config.use_return_dict
#         )

#         outputs = self.deberta(
#             input_ids,
#             token_type_ids=token_type_ids,
#             attention_mask=attention_mask,
#             # position_ids=position_ids,
#             inputs_embeds=inputs_embeds,
#             # output_attentions=output_attentions,
#             # output_hidden_states=output_hidden_states,
#             return_dict=return_dict,
#         )

#         encoder_layer = outputs[0]
#         pooled_output = self.pooler(encoder_layer)
#         pooled_output = self.dropout(pooled_output)
#         logits = self.classifier(pooled_output)

#         loss = None
#         if labels is not None:
#             loss_fn = nn.MSELoss()
#             logits = logits.view(-1).to(labels.dtype)
#             loss = loss_fn(logits, labels.view(-1))
#         # if labels is not None:
#         #     if self.config.problem_type is None:
#         #         if self.num_labels == 1:
#         #             # regression task
#         #             loss_fn = nn.MSELoss()
#         #             logits = logits.view(-1).to(labels.dtype)
#         #             loss = loss_fn(logits, labels.view(-1))
#         #         elif labels.dim() == 1 or labels.size(-1) == 1:
#         #             label_index = (labels >= 0).nonzero()
#         #             labels = labels.long()
#         #             if label_index.size(0) > 0:
#         #                 labeled_logits = torch.gather(
#         #                     logits,
#         #                     0,
#         #                     label_index.expand(label_index.size(0), logits.size(1)),
#         #                 )
#         #                 labels = torch.gather(labels, 0, label_index.view(-1))
#         #                 loss_fct = CrossEntropyLoss()
#         #                 loss = loss_fct(
#         #                     labeled_logits.view(-1, self.num_labels).float(),
#         #                     labels.view(-1),
#         #                 )
#         #             else:
#         #                 loss = torch.tensor(0).to(logits)
#         #         else:
#         #             log_softmax = nn.LogSoftmax(-1)
#         #             loss = -((log_softmax(logits) * labels).sum(-1)).mean()
#         #     elif self.config.problem_type == "regression":
#         #         loss_fct = MSELoss()
#         #         if self.num_labels == 1:
#         #             loss = loss_fct(logits.squeeze(), labels.squeeze())
#         #         else:
#         #             loss = loss_fct(logits, labels)
#         #     elif self.config.problem_type == "single_label_classification":
#         #         loss_fct = CrossEntropyLoss()
#         #         loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
#         #     elif self.config.problem_type == "multi_label_classification":
#         #         loss_fct = BCEWithLogitsLoss()
#         #         loss = loss_fct(logits, labels)
#         # if not return_dict:
#         #     output = (logits,) + outputs[1:]
#         #     return ((loss,) + output) if loss is not None else output

#         return SequenceClassifierOutput(
#             loss=loss,
#             logits=logits,
#             # hidden_states=outputs.hidden_states,
#             # attentions=outputs.attentions,
#         )

In [ ]:
# https://github.com/sinchir0/automated_essay_scoring/blob/main/automated_essay_scoring/exp/exp030.ipynb

# https://dev.classmethod.jp/articles/huggingface-usage-custom-model/
# https://github.com/huggingface/transformers/blob/94b3f544a1f5e04b78d87a2ae32a7ac252e22e31/src/transformers/models/deberta_v2/modeling_deberta_v2.py#L1313
class CustomDebertaSequenceClassification(DebertaV2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)

        #num_labels = getattr(config, "num_labels", 2)
        # self.num_labels = num_labels
        self.num_labels = NUM_LABELS

        self.deberta = DebertaV2Model(config)
        self.pooler = ContextPooler(config)
        output_dim = self.pooler.output_dim

        # self.classifier = nn.Linear(output_dim * 2, num_labels)
        self.classifier = nn.Linear(output_dim * 2, NUM_LABELS)
        drop_out = getattr(config, "cls_dropout", None)
        drop_out = self.config.hidden_dropout_prob if drop_out is None else drop_out
        self.dropout = StableDropout(drop_out)

        # Initialize weights and apply final processing
        self.post_init()

    def get_input_embeddings(self):
        return self.deberta.get_input_embeddings()

    def set_input_embeddings(self, new_embeddings):
        self.deberta.set_input_embeddings(new_embeddings)

    def forward(
        self,
        input_ids_a: torch.Tensor | None = None,
        input_ids_b: torch.Tensor | None = None,
        attention_mask_a: torch.Tensor | None = None,
        attention_mask_b: torch.Tensor | None = None,
        token_type_ids_a: torch.Tensor | None = None,
        token_type_ids_b: torch.Tensor | None = None,
        # position_ids: Optional[torch.Tensor] = None,
        # inputs_embeds_a: torch.Tensor | None = None,
        # inputs_embeds_b: torch.Tensor | None = None,
        labels: torch.Tensor | None = None,
        # output_attentions: Optional[bool] = None,
        # output_hidden_states: Optional[bool] = None,
        return_dict: bool | None = None,
    ) -> tuple | SequenceClassifierOutput:
        return_dict = (
            return_dict if return_dict is not None else self.config.use_return_dict
        )

        outputs_a = self.deberta(
            input_ids_a,
            token_type_ids=token_type_ids_a,
            attention_mask=attention_mask_a,
            return_dict=return_dict,
        )

        outputs_b = self.deberta(
            input_ids_b,
            token_type_ids=token_type_ids_b,
            attention_mask=attention_mask_b,
            return_dict=return_dict,
        )

        encoder_layer_a = outputs_a[0]
        encoder_layer_b = outputs_b[0]

        pooled_output_a = self.pooler(encoder_layer_a)
        pooled_output_b = self.pooler(encoder_layer_b)

        concated_pooled_output = torch.cat((pooled_output_a, pooled_output_b), dim=1)

        logits = self.classifier(concated_pooled_output)

        loss = None

        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    # regression task
                    loss_fn = nn.MSELoss()
                    logits = logits.view(-1).to(labels.dtype)
                    loss = loss_fn(logits, labels.view(-1))
                elif labels.dim() == 1 or labels.size(-1) == 1:
                    label_index = (labels >= 0).nonzero()
                    labels = labels.long()
                    if label_index.size(0) > 0:
                        labeled_logits = torch.gather(
                            logits,
                            0,
                            label_index.expand(label_index.size(0), logits.size(1)),
                        )
                        labels = torch.gather(labels, 0, label_index.view(-1))
                        loss_fct = CrossEntropyLoss()
                        loss = loss_fct(
                            labeled_logits.view(-1, self.num_labels).float(),
                            labels.view(-1),
                        )
                    else:
                        loss = torch.tensor(0).to(logits)
                else:
                    log_softmax = nn.LogSoftmax(-1)
                    loss = -((log_softmax(logits) * labels).sum(-1)).mean()
            elif self.config.problem_type == "regression":
                loss_fct = MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)
        # if not return_dict:
        #     output = (logits,) + outputs[1:]
        #     return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            # hidden_states=outputs.hidden_states,
            # attentions=outputs.attentions,
        )

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME
)
tokenizer.add_tokens(
    [
        AddedToken("\n", normalized=False),
        AddedToken(" " * 2, normalized=False),
    ]
)

class CustomDataCollator:
    def __init__(self, tokenizer):
        self.data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=16)

    def __call__(self, features):
        batch_a = [{'input_ids': f['input_ids_a'], 'token_type_ids': f['token_type_ids_a'], 'attention_mask': f['attention_mask_a']} for f in features]
        batch_b = [{'input_ids': f['input_ids_b'], 'token_type_ids': f['token_type_ids_b'], 'attention_mask': f['attention_mask_b']} for f in features]

        batch_a = self.data_collator(batch_a)
        batch_b = self.data_collator(batch_b)

        labels = torch.tensor([f['labels'] for f in features])

        return {
            'input_ids_a': batch_a['input_ids'],
            'token_type_ids_a': batch_a['token_type_ids'],
            'attention_mask_a': batch_a['attention_mask'],
            'input_ids_b': batch_b['input_ids'],
            'token_type_ids_b': batch_b['token_type_ids'],
            'attention_mask_b': batch_b['attention_mask'],
            'labels': labels
        }

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=16)
data_collator = CustomDataCollator(tokenizer)

# model = AutoModelForSequenceClassification.from_pretrained(
#     MODEL_NAME,
#     num_labels=NUM_LABELS
# )

config = AutoConfig.from_pretrained(MODEL_NAME)

model = CustomDebertaSequenceClassification.from_pretrained(
    MODEL_NAME, config=config
)

model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=16)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

Some weights of CustomDebertaSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(128016, 1024)

# Tokenize

In [ ]:
# def tokenize(examples, max_token_length: int):
#     separator = " [SEP] "

#     joined_text = (
#         examples["last_prompt"]
#         + separator
#         + examples["last_response_a"]
#         + separator
#         + examples["last_response_b"]
#     )

#     tokenized = tokenizer(
#         joined_text,
#         max_length=max_token_length,
#         truncation=True,
#         padding="max_length",
#     )

    # return tokenizer(
    #     joined_text,
    #     max_length=max_token_length,
    #     truncation=True,
    #     padding="max_length",
    # )

def tokenize(examples, suffix, max_token_length: int):
    separator = " [SEP] "

    # TODO: 最後以外の応答も追加できるようにする
    joined_text = (
        examples["last_prompt"]
        + separator
        + examples[f"last_response_{suffix}"]
    )

    tokenized = tokenizer(
        joined_text,
        max_length=max_token_length,
        truncation=True,
        padding="max_length",
    )

    return {key + f"_{suffix}": value for key, value in tokenized.items()}

train_dataset = train_dataset.map(
    tokenize,
    batched=False,
    fn_kwargs={
        "suffix": "a",
        "max_token_length": TRAINING_MAX_LENGTH
    },
    num_proc=NUM_PROC,
).map(
    tokenize,
    batched=False,
    fn_kwargs={
        "suffix": "b",
        "max_token_length": TRAINING_MAX_LENGTH
    },
    num_proc=NUM_PROC,
)

Setting TOKENIZERS_PARALLELISM=false for forked processes.
/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=12):   0%|          | 0/57477 [00:00<?, ? examples/s]

Setting TOKENIZERS_PARALLELISM=false for forked processes.


Map (num_proc=12):   0%|          | 0/57477 [00:00<?, ? examples/s]

In [ ]:
train_dataset

Dataset({
    features: ['id', 'model_a', 'model_b', 'winner_model_a', 'winner_model_b', 'winner_tie', 'len_prompt', 'len_response_a', 'len_response_b', 'last_prompt', 'last_response_a', 'last_response_b', 'label', 'input_ids_a', 'token_type_ids_a', 'attention_mask_a', 'input_ids_b', 'token_type_ids_b', 'attention_mask_b'],
    num_rows: 57477
})

# Train Test Split

In [ ]:
def to_train_valid(ds):
    return DatasetDict({"train": ds["train"], "valid": ds["test"]})

train_dataset = train_dataset.cast_column('label', ClassLabel(num_classes=NUM_LABELS))

train_valid_dataset = to_train_valid(
    (
        train_dataset
        .train_test_split(
            test_size=VALID_DATA_SIZE,
            seed=SEED,
            stratify_by_column="label"
        )
    )
)

Casting the dataset:   0%|          | 0/57477 [00:00<?, ? examples/s]

In [ ]:
train_valid_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'model_a', 'model_b', 'winner_model_a', 'winner_model_b', 'winner_tie', 'len_prompt', 'len_response_a', 'len_response_b', 'last_prompt', 'last_response_a', 'last_response_b', 'label', 'input_ids_a', 'token_type_ids_a', 'attention_mask_a', 'input_ids_b', 'token_type_ids_b', 'attention_mask_b'],
        num_rows: 40233
    })
    valid: Dataset({
        features: ['id', 'model_a', 'model_b', 'winner_model_a', 'winner_model_b', 'winner_tie', 'len_prompt', 'len_response_a', 'len_response_b', 'last_prompt', 'last_response_a', 'last_response_b', 'label', 'input_ids_a', 'token_type_ids_a', 'attention_mask_a', 'input_ids_b', 'token_type_ids_b', 'attention_mask_b'],
        num_rows: 17244
    })
})

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds_prob = softmax(predictions, axis=-1)
    return {"log_loss": log_loss(labels, preds_prob)}

In [ ]:
import torch
from torch.utils.data import Dataset

class ConcatDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, idx):
        item = self.dataset[idx]

        input_ids_a = torch.tensor(item['input_ids_a'])
        token_type_ids_a = torch.tensor(item['token_type_ids_a'])
        attention_mask_a = torch.tensor(item['attention_mask_a'])

        input_ids_b = torch.tensor(item['input_ids_b'])
        token_type_ids_b = torch.tensor(item['token_type_ids_b'])
        attention_mask_b = torch.tensor(item['attention_mask_b'])

        label = torch.tensor(item['label'])

        return {
            'input_ids_a': input_ids_a,
            'token_type_ids_a': token_type_ids_a,
            'attention_mask_a': attention_mask_a,
            'input_ids_b': input_ids_b,
            'token_type_ids_b': token_type_ids_b,
            'attention_mask_b': attention_mask_b,
            'labels': label
        }

    def __len__(self):
        return len(self.dataset)

In [ ]:
# スケジューラの設定
training_args = TrainingArguments(
    output_dir=MODEL_OUTPUT_PATH,
    learning_rate=LR,
    per_device_train_batch_size=TRAIN_BS,
    gradient_accumulation_steps=GRAD_ACC_NUM,
    eval_accumulation_steps=GRAD_ACC_NUM,
    per_device_eval_batch_size=EVAL_BS,
    num_train_epochs=EPOCH,
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=0.1,
    save_strategy="steps",
    save_steps=0.1,
    save_total_limit=1,
    logging_steps=2,
    seed=SEED,
    metric_for_best_model="eval_loss",
    warmup_ratio=0.1,
    lr_scheduler_type="constant_with_warmup",
    report_to=REPORT_TO,
    run_name=EXP_NAME,
    load_best_model_at_end=True,
    fp16=True,
    fp16_full_eval=True,
    gradient_checkpointing=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    # train_dataset=train_valid_dataset["train"],
    train_dataset=ConcatDataset(train_valid_dataset["train"]),
    # eval_dataset=train_valid_dataset["valid"],
    eval_dataset=ConcatDataset(train_valid_dataset["valid"]),
    tokenizer=tokenizer,
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
)

In [ ]:
if TRAINING:
    # モデルの学習
    trainer.train()
    # ログの保存に利用したストレージを削除
    os.system(f"rm -rf {MODEL_OUTPUT_PATH}/checkpoint-*")
    # モデルの保存
    trainer.save_model(MODEL_OUTPUT_PATH)
else:
    # TRAINED_MODEL_PATHを用いて、学習済のモデルを読み込む
    model = AutoModelForSequenceClassification.from_pretrained(
        TRAINED_MODEL_PATH,
        num_labels=NUM_LABELS,
    )

    args = TrainingArguments(
        ".",
        per_device_eval_batch_size=4,
        report_to="none",
        fp16=True,
    )

    trainer = Trainer(
        model=model,
        args=args,
        data_collator=data_collator,
        tokenizer=tokenizer,
    )

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss
1,No log,1.181556
2,1.174500,1.141255
3,1.174500,1.122107


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


# valid_datasetの作成・保存

In [ ]:
# # TRAININGをINFERRENCEでMAX_TOKENを変えるために、validを作り直す
# valid_dataset = train_dataset.filter(
#     lambda example: example["id"]
#     in train_valid_dataset["valid"]["id"],
#     num_proc=NUM_PROC,
# )

# # valid_dataset = valid_dataset.map(
# #     tokenize,
# #     batched=False,
# #     fn_kwargs={"max_token_length": INFERENCE_MAX_LENGTH},
# #     num_proc=NUM_PROC,
# # )

# valid_dataset = valid_dataset.map(
#     tokenize,
#     batched=False,
#     fn_kwargs={
#         "suffix": "a",
#         "max_token_length": INFERENCE_MAX_LENGTH
#     },
#     num_proc=NUM_PROC,
# ).map(
#     tokenize,
#     batched=False,
#     fn_kwargs={
#         "suffix": "b",
#         "max_token_length": INFERENCE_MAX_LENGTH
#     },
#     num_proc=NUM_PROC,
# )

def add_valid_pred(example, idx, valid_pred):
    example["valid_pred"] = valid_pred[idx]
    return example

valid_dataset = train_valid_dataset["valid"]

valid_pred = softmax(
    trainer.predict(ConcatDataset(valid_dataset)).predictions, axis=-1
)

np.save(f"{MODEL_OUTPUT_PATH}/valid_prediction.npy", valid_pred)

valid_dataset = valid_dataset.map(
    add_valid_pred,
    with_indices=True,
    fn_kwargs={"valid_pred": valid_pred}
)

valid_dataset.save_to_disk(f"{MODEL_OUTPUT_PATH}/valid_dataset")

Map:   0%|          | 0/17244 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/17244 [00:00<?, ? examples/s]

# CVの計算

In [ ]:
cv_score = log_loss(valid_dataset["label"], valid_pred)
print(f"CV Score: {cv_score}")

CV Score: 1.1220883316769101


In [ ]:
# output_textを保存
with open(f"{MODEL_OUTPUT_PATH}/cv_score.txt", "w") as f:
    f.write(str(cv_score))

# AWSへのアップロード

In [ ]:
# S3へのアップロード
# TODO: colabでは動かないため直す
if not DEBUG and UPLOAD_DATA_TO_S3:
    # uninstall
    !sudo rm /usr/bin/aws
    !sudo rm /usr/bin/aws_completer
    !sudo rm -rf /usr/local/aws-cli

    # install
    !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
    !unzip -o -qq awscliv2.zip
    !sudo ./aws/install --update

    # upload
    output_name = MODEL_OUTPUT_PATH.split("/")[-1]
    os.system(
        f"aws s3 cp --recursive {MODEL_OUTPUT_PATH} s3://{COMPETITION_NAME}/trained_model/{output_name}"
    )

rm: cannot remove '/usr/bin/aws': No such file or directory
rm: cannot remove '/usr/bin/aws_completer': No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57.9M  100 57.9M    0     0   142M      0 --:--:-- --:--:-- --:--:--  142M
You can now run: /usr/local/bin/aws --version


In [ ]:
# ダウンロード（参考）
# !sudo rm /usr/bin/aws
# !sudo rm /usr/bin/aws_completer
# !sudo rm -rf /usr/local/aws-cli

# !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
# !unzip -o -qq awscliv2.zip
# !sudo ./aws/install --update

# !aws s3 cp --recursive s3://automated-essay-scoring/trained_model/e005-regression /notebooks/automated_essay_scoring/trained_models/e005-regression

# Kaggle Datasetへのupload

In [ ]:
import os

os.system("mkdir -p ~/.kaggle/")
os.system(f"cp /{DATA_PATH}/kaggle.json ~/.kaggle/")
os.system("chmod 600 ~/.kaggle/kaggle.json")

0

In [ ]:
# if not DEBUG and UPLOAD_DATA_TO_KAGGLE:
if UPLOAD_DATA_TO_KAGGLE:
    import os
    import json

    from kaggle.api.kaggle_api_extended import KaggleApi

    def dataset_create_new(dataset_name: str, upload_dir: str):
        # if "_" in dataset_name:
        #     raise ValueError("datasetの名称に_の使用は禁止です")
        dataset_metadata = {}
        dataset_metadata["id"] = f"sinchir0/{dataset_name}"
        dataset_metadata["licenses"] = [{"name": "CC0-1.0"}]
        dataset_metadata["title"] = dataset_name
        with open(os.path.join(upload_dir, "dataset-metadata.json"), "w") as f:
            json.dump(dataset_metadata, f, indent=4)
        api = KaggleApi()
        api.authenticate()
        api.dataset_create_new(folder=upload_dir, convert_to_csv=False, dir_mode="tar")

    print(f"Create Dataset name:{DATASET_NAME}, output_dir:{MODEL_OUTPUT_PATH}")
    dataset_create_new(dataset_name=DATASET_NAME, upload_dir=MODEL_OUTPUT_PATH)

Create Dataset name:e007-use-concat-large-colab-deberta-v3-large, output_dir:/content/drive/MyDrive/Kaggle/lmsys/trained_models/e007-use-concat-large-colab
Starting upload for file config.json


100%|██████████| 884/884 [00:00<00:00, 2.21kB/s]


Upload successful: config.json (884B)
Starting upload for file model.safetensors


100%|██████████| 1.62G/1.62G [00:20<00:00, 86.7MB/s]


Upload successful: model.safetensors (2GB)
Starting upload for file tokenizer_config.json


...

Upload successful: valid_prediction.npy (202KB)
Starting upload for file valid_dataset.tar


100%|██████████| 448M/448M [00:04<00:00, 109MB/s]


Upload successful: valid_dataset.tar (448MB)
Starting upload for file cv_score.txt


100%|██████████| 18.0/18.0 [00:00<00:00, 49.6B/s]


Upload successful: cv_score.txt (18B)


# ローカルからのデータの削除

In [ ]:
if (UPLOAD_DATA_TO_S3 or UPLOAD_DATA_TO_KAGGLE):
    # ローカルからは削除
    os.system(f"rm -rf {MODEL_OUTPUT_PATH}")

In [ ]:
if WANDB:
    wandb.finish()

eval/loss,█▃▁
eval/runtime,██▁
eval/samples_per_second,▁▁█
eval/steps_per_second,▁▁█
test/loss,▁
test/runtime,▁
test/samples_per_second,▁
test/steps_per_second,▁
train/epoch,▁▄▄██
train/global_step,▁▅▅███
train/grad_norm,▁


In [ ]:
print("finish Notebook!")

finish Notebook!
